In [1]:
from dataclasses import dataclass, field
import sys, os
sys.path.insert(1,os.path.abspath('../src'))

Import our code under test.

In [2]:
from fpipeline import pipeline, stepfn, conditionfn, variables, if_, not_, and_, \
    or_, Variable, Attribute, VariableContext, store, eval_vars, \
    list_, dict_, tuple_, set_

Load our Test framework, located in [notebook_reporter.py](./notebook_reporter.py).
It makes use of a simple agnostic [tester.py](./tester.py).

In [3]:
from tester import Test, set_test_reporter

from notebook_reporter import NotebookTestReporter

from collections import namedtuple

REPORTER = NotebookTestReporter()

set_test_reporter(REPORTER)

Test our Test framework

In [4]:
Test("Test Test", None).equals(None)

✅ Test Test: OK

In [5]:
Test("Test Negate Test", None).negate.equals(False)

✅ Test Negate Test: OK

#### Test Data Setup

Our test setup consists of a context class `ctx`, a tracing step function `tstep`, and a predicate `has_value`

Executing a `tstep` adds the arguments to a list in `ctx.trace`. This allows us to verify that the step was called, and that the arguments were properly passed.

The `has_value` condition function compares the value in `ctx.value`, returning `True` if it equals
the supplied value.

The `get_trace` function returns the `ctx.trace` list; it can be used in `.apply` to extract it for comparison.

The `ok` step function just returns `"OK"`.

The `fail` step function just returns `"Failed"`

The `true` condition function returns `True`.

The `false` condition function returns `False`.

In [6]:
@dataclass
class Ctx:
    value: any = None
    result: any = None # Used for testing store()
    trace: list[tuple] = field(default_factory=list)

In [7]:
@stepfn
def tstep(data: Ctx, *args, **kwargs):
    d = (*args, {**kwargs})
    data.trace.append(d)
    return data

def get_trace(data: Ctx):
    return data.trace

@stepfn
def ok(_: Ctx):
    return 'OK'

@stepfn
def fail(_: Ctx):
    return "Failed"

In [8]:
@conditionfn
def has_value(data: Ctx, value):
    return data.value == value

@conditionfn
def true(_: Ctx):
    return True

@conditionfn
def false(_: Ctx):
    return False

> Debugging note: inserting `.raise_exception` into a `Test` call chain will cause erroring tests
to raise the original exception. `Test(`_..._`handle_exceptions=False)` will turn off exception handling entirely.

#### Test Cases

In [9]:
Test("@stepfn step takes a single argument",
    tstep(7, 8).__code__.co_argcount
).equals(1)

✅ @stepfn step takes a single argument: OK

In [10]:
Test("@conditionfn condition takes a single argument",
    has_value(7, 8).__code__.co_argcount
).equals(1)

✅ @conditionfn condition takes a single argument: OK

In [11]:
Test("@stepfn capture args",
    tstep(7, 5)(Ctx()).trace
).equals([(7, 5, {})])

✅ @stepfn capture args: OK

In [12]:
Test("@conditionfn capture args (true)",
    has_value(7)
).apply(Ctx(7)).is_true()

✅ @conditionfn capture args (true): OK

In [13]:
Test("@conditionfn capture args (false)",
    has_value(7)
).apply(Ctx()).is_false()

✅ @conditionfn capture args (false): OK

In [14]:
Test("@stepfn name",
    tstep.__name__
).equals("tstep")

✅ @stepfn name: OK

In [15]:
Test("@conditionfn name",
    has_value.__name__
).equals("has_value")

✅ @conditionfn name: OK

In [16]:
Test("Simple pipeline",
    pipeline(
        tstep(7),
        tstep(78)
        )
 ).apply(Ctx())\
    .call(get_trace)\
    .equals([
            (7, {}),
            (78, {})
        ])

✅ Simple pipeline: OK

#### Conditional Tests

In [17]:
Test("if_ True",
    if_(
        true(),
        ok()
        )
).apply(Ctx())\
.equals('OK')

✅ if_ True: OK

In [18]:
Test("if_ False",
     if_(
        false(),
        ok()
     )
).apply(Ctx())\
.equals(None)


✅ if_ False: OK

In [19]:
Test("if_ True else",
    if_(
        true(),
        ok(),
        fail()
    )
).apply(Ctx(42))\
.equals('OK')

✅ if_ True else: OK

In [20]:
Test("if_ False else",
     if_(
         false(),
         fail(),
         ok()
     )
     ).apply(Ctx())\
        .equals('OK')


✅ if_ False else: OK

In [21]:
Test("not_ True",
    not_(true())
).apply(Ctx())\
    .equals(False)

✅ not_ True: OK

In [22]:
Test("not_ False",
     not_(false())
     ).apply(Ctx())\
          .equals(True)

✅ not_ False: OK

In [23]:
Test("or_ 0",
or_()
).apply(Ctx())\
    .equals(False)

✅ or_ 0: OK

In [24]:
Test("or_ T",
     or_(true())
     ).apply(Ctx())\
          .equals(True)


✅ or_ T: OK

In [25]:
Test("or_ F",
    or_(false())
).apply(Ctx())\
    .equals(False)

✅ or_ F: OK

In [26]:
Test("or_ F T",
     or_(false(), true())
     ).apply(Ctx())\
          .equals(True)

✅ or_ F T: OK

In [27]:
Test("or_ T F",
     or_(true(), false())
     ).apply(Ctx()).equals(True)

✅ or_ T F: OK

In [28]:
Test("or_ T T",
     or_(true(), true())
).apply(Ctx())\
     .equals(True)

✅ or_ T T: OK

In [29]:
Test("and_ 0",
    and_()
).apply(Ctx())\
    .equals(True)

✅ and_ 0: OK

In [30]:
Test("and_ T",
     and_(true())
).apply(Ctx())\
     .equals(True)

✅ and_ T: OK

In [31]:
Test("and_ F",
     and_(false())
).apply(Ctx())\
     .equals(False)

✅ and_ F: OK

In [32]:
Test("and_ T F",
    and_(true(), false())
).apply(Ctx())\
    .equals(False)

✅ and_ T F: OK

In [33]:
Test("and_ F T",
     and_(false(), true())
).apply(Ctx())\
     .equals(False)

✅ and_ F T: OK

In [34]:
Test("and_ T T",
    and_(true(), true())
).apply(Ctx())\
    .equals(True)

✅ and_ T T: OK

In [35]:
Test("and_ F F",
     and_(false(), false())
).apply(Ctx())\
     .equals(False)

✅ and_ F F: OK

#### Variable Tests

In [36]:
@stepfn
def f0(ctx: Ctx):
    with variables(ctx) as vars:
        return vars
Test('Close vars',
    f0()
).apply(Ctx())\
    .call(lambda c: c.closed)\
    .is_true()

✅ Close vars: OK

In [37]:
@stepfn
def f1(ctx: Ctx):
    with variables(ctx) as vars:
        v1, v2 = vars.variable('v1', 'v2')
        a1, a2 = vars.attribute('value', 'a2')
        return vars
Test('Clear vars',
     f1()
).apply(Ctx())\
    .call(lambda c: len(c._variables))\
        .equals(0)

✅ Clear vars: OK

In [38]:
@stepfn
def f2(ctx: Ctx):
    c = VariableContext(ctx)
    v = c.variable('v')
    a = c.attribute('a')
    c.close()
Test("Check vars lifecycle",
    f2()
).apply(Ctx())\
    .equals(None)

✅ Check vars lifecycle: OK

In [39]:
@stepfn
def f3(ctx: Ctx):
    with variables(ctx) as vars:
        v1, v2 = vars.variable('v1', 'v2')
        a1, a2 = vars.attribute('a1', 'a2')
        return list(vars._variables.keys())

Test('Check vars recorded',
     f3()
).apply(Ctx())\
    .equals(['v1', 'v2', 'a1', 'a2'])


✅ Check vars recorded: OK

In [40]:
@stepfn
def f4(ctx: Ctx):
    with variables(ctx) as vars:
        a = vars.attribute('value')
        return a.value
Test("Attribute.value",
    f4()
).apply(Ctx(55))\
    .equals(55)

✅ Attribute.value: OK

In [41]:
# Attributes should not be returned from steps.
@stepfn
def f5(ctx: Ctx):
    with variables(ctx) as vars:
        a = vars.attribute('value')
        return a
Test("Attribute return",
    f5()
).apply(Ctx(55))\
    .is_exception()

✅ Attribute return: OK

In [42]:
@stepfn
def f6(ctx: Ctx):
    with variables(ctx) as vars:
        v1, v2 = vars.variable('v1', 'v2')
        v1.value = 'a'
        v2.value = v1.value
        return v2.value
Test('Variables',
    f6()
).apply(Ctx())\
    .equals('a')

✅ Variables: OK

In [43]:
# Variables should not be returned from steps
@stepfn
def f7(ctx: Ctx):
    with variables(ctx) as vars:
        v = vars.variable('value')
        return v

Test("Variable return forbidden",
     f7()
).apply(Ctx(55))\
    .is_exception()


✅ Variable return forbidden: OK

In [44]:
@stepfn
def f8(ctx: Ctx):
    with variables(ctx) as vars:
        v = vars.variable('v')
        v.value = 99
        return vars.pipeline(v)
Test('Variable Pipeline return variable',
    f8()
).apply(Ctx())\
    .equals(99)

✅ Variable Pipeline return variable: OK

In [45]:
@stepfn
def f9(ctx: Ctx):
    with variables(ctx) as vars:
        v, r = vars.attribute('value', 'result')
        return vars.pipeline(
            store(r, v)
        )
Test('Store',
    f9()
).apply(Ctx(72))\
    .equals(72)

✅ Store: OK

In [46]:
@stepfn
def f10(ctx: Ctx):
    with variables(ctx) as vars:
        v, r = vars.attribute('value', 'result')
        return vars.pipeline(
            store(r, v),
            lambda data: data # Return the context data for examination
        )
Test('Store ctx.result',
    f10()
).apply(Ctx(72))\
    .attribute('result')\
        .equals(72)

✅ Store ctx.result: OK

### Test eval_vars

In [47]:
Test("eval_vars simple",
    eval_vars({}, [3, {'a': 5}, (7, 3)])
).equals([3, {'a': 5}, (7, 3)])

✅ eval_vars simple: OK

In [48]:
vv = Variable('vv')
vv.value = 77
Test("eval_vars variable", eval_vars({}, vv))\
    .equals(77)

✅ eval_vars variable: OK

In [49]:
snt = Variable('snt')
snt.value = 77
tup = namedtuple('typ', ['x', 'y'])
Test("eval_vars namedtuple", eval_vars({}, tup(3, snt)))\
    .equals(tup(3, 77))

✅ eval_vars namedtuple: OK

In [50]:
vset = Variable('vset')
vset.value = 42
Test("eval_vars set", eval_vars({},{8, vset, 42}))\
    .equals({8, 42})

✅ eval_vars set: OK

In [51]:
@stepfn
def f11(ctx: Ctx):
    with variables(ctx) as vars:
        x, y = vars.attribute('x', 'y')
        return vars.pipeline(lambda data: (y, x))
Test('Pipeline struct return',
    f11()
).apply({'x': 5, 'y': 7})\
    .equals((7, 5))

✅ Pipeline struct return: OK

In [52]:
REPORTER.report(columns=3)

✅ Test Test: OK,✅ Test Negate Test: OK,✅ @stepfn step takes a single argument: OK
✅ @conditionfn condition takes a single argument: OK,✅ @stepfn capture args: OK,✅ @conditionfn capture args (true): OK
✅ @conditionfn capture args (false): OK,✅ @stepfn name: OK,✅ @conditionfn name: OK
✅ Simple pipeline: OK,✅ if_ True: OK,✅ if_ False: OK
✅ if_ True else: OK,✅ if_ False else: OK,✅ not_ True: OK
✅ not_ False: OK,✅ or_ 0: OK,✅ or_ T: OK
✅ or_ F: OK,✅ or_ F T: OK,✅ or_ T F: OK
✅ or_ T T: OK,✅ and_ 0: OK,✅ and_ T: OK
✅ and_ F: OK,✅ and_ T F: OK,✅ and_ F T: OK
✅ and_ T T: OK,✅ and_ F F: OK,✅ Close vars: OK
✅ Clear vars: OK,✅ Check vars lifecycle: OK,✅ Check vars recorded: OK
